In [28]:
import network as net
from config import config
import torch
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torchvision import utils as vutils
import random

seed = 7412

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

def save_tile_images(resl, images, save_path):
    images = images.detach().cpu()
    image_size = 2**resl
    vutils.save_image(images, save_path, nrow=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# root = '../storage/PGGAN/results/emarie_rose/2019_10_02_15_48_14'
# root = '../storage/PGGAN/results/emarie_rose/2019_10_02_15_48_59'
#root = '../storage/PGGAN/results/emarie_rose/2019_10_02_15_49_29'
root = '../storage/PGGAN/results/emarie_rose/2019_10_10_13_34_02'

model_root = os.path.join(root, 'models')
save_root = os.path.join(root, 'tile_images')

model_list = os.listdir(model_root)
model_list = [m for m in model_list  if 'gen' in m]
model_list = sorted(model_list, key=lambda x: int(x[8:].split('.')[0]))
print(model_list)

G = net.Generator(config).to(device)
G.eval()
z = torch.FloatTensor(100, config.nz).normal_(0.0, 1.0).to(device)

with torch.no_grad():
    old_resl = 2
    for m in model_list:
        resl = int(m[5])
        if resl > old_resl:
            G.grow_network(resl)
            G.flush_network()
            G.to(device)
            G.eval()
            old_resl = resl
        checkpoint = torch.load(os.path.join(model_root, m))
        G.load_state_dict(checkpoint['state_dict'])

#         z = torch.FloatTensor(100, config.nz).normal_(0.0, 1.0).to(device)
        images1 = G(z[:50]).detach().cpu()
        images2 = G(z[50:]).detach().cpu()
        images = torch.cat((images1, images2), 0)
        os.makedirs(save_root, exist_ok=True)
        save_tile_images(resl, images, os.path.join(save_root, m+'.png'))
        
        images = None
        for i in range(10):
            start_ind = i*10
            end_ind = i*10 + 9
            zs = torch.zeros(10, config.nz).to(device)

            diff = z[end_ind] - z[start_ind]
            for j in range(9+1):
                zs[j] = z[start_ind] + j * diff / 9
            tmp = G(zs).detach().cpu()
            if images is None:
                images = tmp
            else:
                images = torch.cat((images, tmp), 0)
            save_tile_images(resl, images, os.path.join(save_root, 'int_'+m+'.png'))

['gen_R3_T155.pth.tar', 'gen_R3_T160.pth.tar', 'gen_R3_T165.pth.tar', 'gen_R3_T170.pth.tar', 'gen_R3_T175.pth.tar', 'gen_R3_T180.pth.tar', 'gen_R3_T185.pth.tar', 'gen_R3_T190.pth.tar', 'gen_R3_T195.pth.tar', 'gen_R3_T200.pth.tar', 'gen_R4_T255.pth.tar', 'gen_R4_T260.pth.tar', 'gen_R4_T265.pth.tar', 'gen_R4_T270.pth.tar', 'gen_R4_T275.pth.tar', 'gen_R4_T280.pth.tar', 'gen_R4_T285.pth.tar', 'gen_R4_T290.pth.tar', 'gen_R4_T295.pth.tar', 'gen_R4_T300.pth.tar', 'gen_R5_T355.pth.tar', 'gen_R5_T360.pth.tar', 'gen_R5_T365.pth.tar', 'gen_R5_T370.pth.tar', 'gen_R5_T375.pth.tar', 'gen_R5_T380.pth.tar', 'gen_R5_T385.pth.tar', 'gen_R5_T390.pth.tar', 'gen_R5_T395.pth.tar', 'gen_R5_T400.pth.tar', 'gen_R6_T455.pth.tar', 'gen_R6_T460.pth.tar', 'gen_R6_T465.pth.tar', 'gen_R6_T470.pth.tar', 'gen_R6_T475.pth.tar', 'gen_R6_T480.pth.tar', 'gen_R6_T485.pth.tar', 'gen_R6_T490.pth.tar', 'gen_R6_T495.pth.tar', 'gen_R6_T500.pth.tar', 'gen_R7_T555.pth.tar', 'gen_R7_T560.pth.tar', 'gen_R7_T565.pth.tar', 'gen_R7_T5

/home/tacchan7412/Developer/pggan-pytorch/custom_layers.py:104: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if initializer == 'kaiming':    kaiming_normal(self.conv.weight, a=calculate_gain('conv2d'))


growing network[4x4 to 8x8]. It may take few seconds...
flushing network... It may take few seconds...


/home/tacchan7412/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


growing network[8x8 to 16x16]. It may take few seconds...
flushing network... It may take few seconds...
growing network[16x16 to 32x32]. It may take few seconds...
flushing network... It may take few seconds...
growing network[32x32 to 64x64]. It may take few seconds...
flushing network... It may take few seconds...
growing network[64x64 to 128x128]. It may take few seconds...
flushing network... It may take few seconds...
growing network[128x128 to 256x256]. It may take few seconds...
flushing network... It may take few seconds...
growing network[256x256 to 512x512]. It may take few seconds...
flushing network... It may take few seconds...


KeyboardInterrupt: 

# interpolation animation
- 10枚
- 遷移に20frame
- 戻ってくる

In [26]:
import network as net
from config import config
import torch
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torchvision import utils as vutils
import torchvision.transforms.functional as TF
import random

seed = 7412

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

root = '../storage/PGGAN/results/emarie_rose/2019_10_02_15_48_14'
# root = '../storage/PGGAN/results/emarie_rose/2019_10_02_15_48_59'
#root = '../storage/PGGAN/results/emarie_rose/2019_10_02_15_49_29'

model_root = os.path.join(root, 'models')

checkpoint = torch.load(os.path.join(model_root, 'gen_R9_T950.pth.tar'))

G = net.Generator(config).to(device)
G.eval()
z = torch.FloatTensor(100, config.nz).normal_(0.0, 1.0).to(device)

resl = 9
frame = 20
tiles = 10
with torch.no_grad():
    for i in range(resl - 1):
        G.grow_network(i+2)
        G.flush_network()
        G.to(device)
        G.eval()
    G.load_state_dict(checkpoint['state_dict'])
#     for _ in range(10):
#         z = torch.FloatTensor(100, config.nz).normal_(0.0, 1.0).to(device)
    
    images = None
    for i in range(tiles):
            start_ind = i % tiles
            end_ind = (i+1) % tiles
            print(start_ind, end_ind)
            zs = torch.zeros(frame, config.nz).to(device)

            diff = z[end_ind] - z[start_ind]
            for j in range(frame):
                zs[j] = z[start_ind] + j * diff / (frame-1)
            tmp = G(zs).detach().cpu()
            if images is None:
                images = tmp
            else:
                images = torch.cat((images, tmp), 0)
    pil_images = []
    for im in images:
        im = im.mul_(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
        pil_images.append(Image.fromarray(im))

print(len(pil_images))
pil_images[0].save('tmp.gif', save_all=True, append_images=pil_images[1:], optimize=False, duration=200, loop=0)

/home/tacchan7412/Developer/pggan-pytorch/custom_layers.py:104: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if initializer == 'kaiming':    kaiming_normal(self.conv.weight, a=calculate_gain('conv2d'))


flushing network... It may take few seconds...
growing network[4x4 to 8x8]. It may take few seconds...
flushing network... It may take few seconds...
growing network[8x8 to 16x16]. It may take few seconds...
flushing network... It may take few seconds...
growing network[16x16 to 32x32]. It may take few seconds...
flushing network... It may take few seconds...
growing network[32x32 to 64x64]. It may take few seconds...
flushing network... It may take few seconds...
growing network[64x64 to 128x128]. It may take few seconds...
flushing network... It may take few seconds...
growing network[128x128 to 256x256]. It may take few seconds...
flushing network... It may take few seconds...
growing network[256x256 to 512x512]. It may take few seconds...
flushing network... It may take few seconds...
0 1


/home/tacchan7412/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 0
200
